# Tools


In [ ]:
import { WikipediaQueryRun } from "npm:langchain/tools";

const tool = new WikipediaQueryRun({
  topKResults: 3,
  maxDocContentLength: 4000,
});

const res = await tool.call("Pizza");

console.log(res);

In [1]:
import { WebBrowser } from "langchain/tools/webbrowser";
import { ChatOpenAI } from "langchain/chat_models/openai";
import { OpenAIEmbeddings } from "langchain/embeddings/openai";

export async function run() {
  // this will not work with Azure OpenAI API yet
  // Azure OpenAI API does not support embedding with multiple inputs yet
  // Too many inputs. The max number of inputs is 1.  We hope to increase the number of inputs per request soon. Please contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 for further questions.
  // So we will fail fast, when Azure OpenAI API is used
  if (process.env.AZURE_OPENAI_API_KEY) {
    throw new Error(
      "Azure OpenAI API does not support embedding with multiple inputs yet"
    );
  }

  const model = new ChatOpenAI({ temperature: 0 });
  const embeddings = new OpenAIEmbeddings(
    process.env.AZURE_OPENAI_API_KEY
      ? { azureOpenAIApiDeploymentName: "Embeddings2" }
      : {}
  );

  const browser = new WebBrowser({ model, embeddings });

  const result = await browser.call(
    `"https://react.dev/blog/2023/05/03/react-canaries","What is React Canary?"`
  );

  console.log(result);
}

In [ ]:
import { OpenAI } from "npm:langchain/llms/openai";
import { SqlDatabase } from "npm:langchain/sql_db";
import { createSqlAgent, SqlToolkit } from "npm:langchain/agents/toolkits/sql";
import { DataSource } from "npm:typeorm";

/** This example uses Chinook database, which is a sample database available for SQL Server, Oracle, MySQL, etc.
 * To set it up follow the instructions on https://database.guide/2-sample-databases-sqlite/, placing the .db file
 * in the examples folder.
 */
export const run = async () => {
  const datasource = new DataSource({
    type: "sqlite",
    database: "Chinook.db",
  });
  const db = await SqlDatabase.fromDataSourceParams({
    appDataSource: datasource,
  });
  const model = new OpenAI({ temperature: 0 });
  const toolkit = new SqlToolkit(db, model);
  const executor = createSqlAgent(model, toolkit);

  const input = `List the total sales per country. Which country's customers spent the most?`;

  console.log(`Executing with input "${input}"...`);

  const result = await executor.invoke({ input });

  console.log(`Got output ${result.output}`);

  console.log(
    `Got intermediate steps ${JSON.stringify(
      result.intermediateSteps,
      null,
      2
    )}`
  );

  await datasource.destroy();
};

await run();

## Agents Life Cycle

In [ ]:
import { initializeAgentExecutorWithOptions } from "npm:langchain/agents";
import { OpenAI } from "npm:langchain/llms/openai";
import { WikipediaQueryRun } from "npm:langchain/tools";

const model = new OpenAI({ temperature: 0 });
const tools = [
  new WikipediaQueryRun({
    topKResults: 1,
    maxDocContentLength: 1000,
  })
];
const executor = await initializeAgentExecutorWithOptions(tools, model);

const input = `Who is John Lennon?`;
const result = await executor.invoke(
  { input },
  {
    callbacks: [
      {
        handleAgentAction(action, runId) {
          console.log("\nhandleAgentAction", action, runId);
        },
        handleAgentEnd(action, runId) {
          console.log("\nhandleAgentEnd", action, runId);
        },
        handleToolEnd(output, runId) {
          console.log("\nhandleToolEnd", output, runId);
        },
      },
    ],
  }
);

console.log({ result });

In [ ]:
import { initializeAgentExecutorWithOptions } from "npm:langchain/agents";
import { OpenAI } from "npm:langchain/llms/openai";
import { GoogleCustomSearch  } from "npm:langchain/tools";
import { Calculator } from "npm:langchain/tools/calculator";

const model = new OpenAI({ temperature: 0 });
const tools = [
  new GoogleCustomSearch (
  ),
  new Calculator(),
];
const executor = await initializeAgentExecutorWithOptions(tools, model);

const input = `What is the age of John Lennon? What is his age mulitply to 2?`;
const result = await executor.invoke(
  { input }
);

console.log({ result });